In [ ]:
VECTOR_PATH = '../glove.6B.200d.pkl'
ARTICLES_PATH = '../articles/'
GOLD_SUMMARIES_PATH = '../gold_summaries/'

Takes ~37 seconds in my computer

In [ ]:
import pickle
import numpy as np
import os
import re
from rouge import Rouge 
import collections, functools, operator 

ALL_FILES = os.listdir(os.path.abspath( os.path.join( os.getcwd() , ARTICLES_PATH )))

vec = pickle.load(open(VECTOR_PATH , 'rb'))

punc = {'£': ' ' , '"': ' ', '!': ' ', '^': ' ', '%': ' ', '<': ' ', '+': ' ', '~': ' ', '*': ' ', ';': ' ', ':': ' ', '(': ' ', '?': ' ', '&': ' ', '}': ' ', ']': ' ', '|': ' ', ',': ' ', "'": ' ', ')': ' ', '-': ' ', '#': ' ', '`': ' ', '@': ' ', '/': ' ', '$': ' ', '_': ' ', '{': ' ', '.': ' ', '>': ' ', '[': ' ', '\\': ' ', '=': ' '}
punc = str.maketrans(punc)

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def preprocess(string): # Removes punctuations, replaces them with space character
    global punc
    string = string.translate(punc)
    return string.lower().strip()

In [ ]:
def parse_documents_to_sentences():
    docs_by_sentences = []
    for filename in file_list: # Read every document
        f = open(os.path.abspath(os.path.join(os.getcwd() , ARTICLES_PATH , filename)) , 'r')
        title = f.readline() # Get title as a sentence
        f = f.read() # Read rest of the document
        sentences = [title.strip()]
        sentences.extend(split_into_sentences(f)) # Extract sentences
        docs_by_sentences.append(sentences)
    return docs_by_sentences

I use two different model when representing sentences. First model simply take avarages of words while second one considers in term frequency for weighted avarage. Below one is the simple one.

In [ ]:
def sent2vec(docs_by_sentences):
    docs_as_vectors = []
    for doc in docs_by_sentences:
        vectors = []
        for sentence in doc:
            vector = np.zeros(200) # initialize vector represents the sentece
            number_of_words = 0 # Number of valid words in the sentence
            for word in preprocess(sentence).split():
                if vec.get(word) is not None: # If the word has a vector representation 
                    vector = vector + vec[word]
                    number_of_words += 1
            if number_of_words == 0: # If there is no valid word in sentence, for example there are sentences like '.' because of edge cases in preprocess, simply ignore it
                vectors.append("PASS")
                #print("Oopss this is not a sentence: " ,  sentence)
            else:
                vectors.append(vector/number_of_words) # Sentence is avarage of words
        docs_as_vectors.append(vectors)
    return docs_as_vectors

This is model with tf's. I multiply every word vector to its term frequency.

In [ ]:
# def sent2vec(docs_by_sentences):
#     docs_as_vectors = []
#     for doc in docs_by_sentences:
#         tf = {}
#         for sentence in doc:
#             for word in preprocess(sentence).split():
#                 if vec.get(word) is not None:
#                     tf[word] = tf.get(word , 0.0) + 1.0
#         vectors = []
#         total_words = sum(tf.values())
#         for sentence in doc:
#             vector = np.zeros(200) # initialize vector represents the sentece
#             number_of_words = 0
#             for word in preprocess(sentence).split():
#                 if vec.get(word) is not None: # If the word has a vector representation 
#                     vector = vector + np.multiply(vec[word] ,  (tf[word]/total_words))
#                     number_of_words += 1
#             if number_of_words == 0: # If there is no valid word in sentence, for example there are sentences like '.' because of edge cases in preprocess, simply ignore it
#                 vectors.append("PASS")
#                 #print("Oopss this is not a sentence: " ,  sentence)
#             else:
#                 vectors.append(vector) # Sentence is avarage of words
#         docs_as_vectors.append(vectors)
#     return docs_as_vectors


I use static K for K-Means. I use first K point as initial values.

In [ ]:
def cluster(docs_as_vectors):
    extracted_summaries = []
    for i in range(len(docs_as_vectors)):
        #print(file_list[i])
        sentences = kmean(docs_as_vectors[i]) # Take positions from k mean
        summary = ""
        for sentence in sentences:
            summary += docs_by_sentences[i][sentence] # Get sentences from document and create summary
            summary += " "
        extracted_summaries.append(summary.strip())
    return extracted_summaries

def kmean(vectors): # Returns the positions of sentences that will be used in summary for a document
    K = 3
    count = 0
    means = []
    for i in range(len(vectors)): # Initial means, I choose first K, skip empty sentences
        if type(vectors[i]) == str:
            continue
        count += 1
        means.append(vectors[i])
        if count == K:
            break
    clusters = np.zeros(len(vectors) , dtype=np.uint8) # clusters[i] means sentence i is in cluster[i]
    for i in range(len(vectors)):
        vector = vectors[i]
        if type(vector) == str: # I push "PASS" as vector for invalid sentences so I check if it a vector or string
            continue
        clusters[i] = np.argmin([np.linalg.norm(vector - mean) for mean in means]) + 1 # assign each sentence a new cluster according current means
    new_mean = [np.mean([vectors[i] for i in range(len(vectors)) if clusters[i] == k] , axis=0) for k in range(1,K+1)] # Re-calculate current means of clusters
    while np.mean([np.linalg.norm(means[i] - new_mean[i]) for i in range(len(means))]) > 0.000001: # Until means don't change much
        #print("step ..")
        means = new_mean
        clusters = np.zeros(len(vectors) , dtype=np.uint8)
        for i in range(len(vectors)):
            vector = vectors[i]
            if type(vector) == str:
                continue
            clusters[i] = np.argmin([np.linalg.norm(vector - mean) for mean in means]) + 1 # Reassign sentences to new clusters
        new_mean = [np.mean([vectors[i] for i in range(len(vectors)) if clusters[i] == k] , axis=0) for k in range(1,K+1)] # Recalculate cluster's means
    means = new_mean
    #print(clusters)
    res = []
    for i in range(1,K+1): # Find the sentence that is closest to the mean for every cluster and choose them as summary sentences
        pos = -1
        dist = np.inf
        for j in range(0,len(clusters)):
            if clusters[j] == i:
                if np.linalg.norm(vectors[j] - means[i-1]) < dist:
                    dist = np.linalg.norm(vectors[j] - means[i-1])
                    pos = j
        res.append(pos)
    return res

In [ ]:
def evaluation(gold_summaries, extracted_summaries):
    rouge = Rouge()
    rouge_scores = rouge.get_scores(extracted_summaries , gold_summaries , avg=True)
    return rouge_scores

In [ ]:
files_fold = np.array_split(ALL_FILES , 10) # K-fold, I use k = 10. Use first fold as test, others are for training

file_list = files_fold[0]
docs_by_sentences = parse_documents_to_sentences()
docs_as_vectors = sent2vec(docs_by_sentences)
extracted_summaries = cluster(docs_as_vectors)

gold_summaries = []
for filename in file_list:
    f = open(os.path.abspath(os.path.join(os.getcwd() , GOLD_SUMMARIES_PATH , filename)) , 'r').read()
    gold_summaries.append(f)

scores = evaluation(gold_summaries , extracted_summaries)
print("------ TEST SCORES ------")
print(scores)
# Calculate validation scores
validate_scores_one = [] 
validate_scores_two = []
validate_scores_l = []
for file_list in files_fold[1:]:
    docs_by_sentences = parse_documents_to_sentences()
    docs_as_vectors = sent2vec(docs_by_sentences)
    extracted_summaries = cluster(docs_as_vectors)

    gold_summaries = []
    for filename in file_list:
        f = open(os.path.abspath(os.path.join(os.getcwd() , GOLD_SUMMARIES_PATH , filename)) , 'r').read()
        gold_summaries.append(f)

    scores = evaluation(gold_summaries , extracted_summaries)
    validate_scores_one.append(scores['rouge-1'])
    validate_scores_l.append(scores['rouge-2'])
    validate_scores_two.append(scores['rouge-l'])

validate_means = { 'rouge-1': dict(functools.reduce(operator.add, 
         map(collections.Counter, validate_scores_one))),
            'rouge-2': dict(functools.reduce(operator.add, 
         map(collections.Counter, validate_scores_two))),
            'rouge-l': dict(functools.reduce(operator.add, 
         map(collections.Counter, validate_scores_l)))}

for key in validate_means.keys(): # First I calculate sums, now I divide all values to 9 to calculate mean
    for key_ in validate_means[key].keys():
        validate_means[key][key_] = validate_means[key][key_] / 9.0

print("------ VALIDATION SCORES ------")
print("means ==> " , validate_means)

validate_var = {}
squares = [] # Stores squares of (val - mean)**2, them sum all of them and divide 9 to calculate variation
mean = validate_means['rouge-1']
for score in validate_scores_one:
    squares.append({
        'f': (score['f'] - mean['f'])**2,
        'p': (score['p'] - mean['p'])**2,
        'r': (score['r'] - mean['r'])**2
    })
validate_var['rouge-1'] = dict(functools.reduce(operator.add, 
         map(collections.Counter, squares)))
squares = []
mean = validate_means['rouge-2']
for score in validate_scores_one:
    squares.append({
        'f': (score['f'] - mean['f'])**2,
        'p': (score['p'] - mean['p'])**2,
        'r': (score['r'] - mean['r'])**2
    })
validate_var['rouge-2'] = dict(functools.reduce(operator.add, 
         map(collections.Counter, squares)))
squares = []
mean = validate_means['rouge-l']
for score in validate_scores_one:
    squares.append({
        'f': (score['f'] - mean['f'])**2,
        'p': (score['p'] - mean['p'])**2,
        'r': (score['r'] - mean['r'])**2
    })
validate_var['rouge-l'] = dict(functools.reduce(operator.add, 
         map(collections.Counter, squares)))

for key in validate_var.keys():
    for key_ in validate_var[key].keys():
        validate_var[key][key_] = validate_var[key][key_] / 9.0

print("variations ==> " , validate_var)

Model 1: <br>
------ TEST SCORES ------ <br>
{'rouge-1': {'f': 0.4803040479084102, 'p': 0.7232442115995087, 'r': 0.36760463219012707}, 'rouge-2': {'f': 0.35406512941748186, 'p': 0.5956987919724204, 'r': 0.2585699686276442}, 'rouge-l': {'f': 0.39629701595657746, 'p': 0.7077406122600427, 'r': 0.3595953228819612}} <br>
------ VALIDATION SCORES ------ <br>
means ==>  {'rouge-1': {'f': 0.45293486430462093, 'p': 0.7337109698177251, 'r': 0.33966474903256505}, 'rouge-2': {'f': 0.36218016025752886, 'p': 0.7158007184645797, 'r': 0.33119691474537416}, 'rouge-l': {'f': 0.32979074765465355, 'p': 0.6057084485814125, 'r': 0.2369253106378605}} <br>
variations ==>  {'rouge-1': {'f': 0.0007295177929322034, 'p': 0.00033214824205945727, 'r': 0.00085176714178182}, 'rouge-2': {'f': 0.008965934099607477, 'p': 0.0006529253455923054, 'r': 0.000923471359297147}, 'rouge-l': {'f': 0.01589399125843298, 'p': 0.01671679368491213, 'r': 0.01140715934344113}} <br>

Model 2: <br>
------ TEST SCORES ------ <br>
{'rouge-1': {'f': 0.419511775717035, 'p': 0.6278524033801669, 'r': 0.3230158028078079}, 'rouge-2': {'f': 0.2818223155601035, 'p': 0.4720717750614849, 'r': 0.2073344441552195}, 'rouge-l': {'f': 0.3444399179703348, 'p': 0.6080876388502864, 'r': 0.3130674264485994}} <br>
------ VALIDATION SCORES ------ <br>
means ==>  {'rouge-1': {'f': 0.38534570272519736, 'p': 0.6268919714401204, 'r': 0.28980828246072077}, 'rouge-2': {'f': 0.30576449426104646, 'p': 0.6053994427906307, 'r': 0.28027556399447767}, 'rouge-l': {'f': 0.2531938092218845, 'p': 0.46190996654299915, 'r': 0.18329398628165158}} <br>
variations ==>  {'rouge-1': {'f': 0.0006976009215804269, 'p': 0.000165205406525276, 'r': 0.0007658420611953346}, 'rouge-2': {'f': 0.007030769662195063, 'p': 0.0006271341942744075, 'r': 0.0008567147825519877}, 'rouge-l': {'f': 0.018161723878091358, 'p': 0.027384267346398997, 'r': 0.012111137351717819}} <br>

I choose model 1 because scores are higher and yet it is simple